<img src="logoUGA.png"></img>

# Cours 3 : Déclarativité et Contrôle

Séance 4 et 5

Fin du prolog pur, début de la convergence entre différents paradigmes de programmation (CP, RO, CAS), avec la logique comme pilier central général.

1) Réversibilité (quand on peut, si on peut : ce n'est pas un objectif en soi ; mais parfois, c'est une partie de la solution, cela ouvre d'autres perspectives de résolution, à explorer !)
   * Prédicat naturellement symétrique
   * Cas de la concaténation (8 cas à observer)
   * Réversibilité de prédicats courants ?
3) Contraintes (réversibilité arithmétique ?)
   * cas du dif, cas du is
   * cas des contraintes numériques (linéaires et/ou pas, entière vs réelle vs complexe)
   * cas des contraintes structurelles (=>graphes !)
   * cas des contraintes ensemblistes
4) Non-déterminisme
   * Solutions redondantes
   * Solutions non-redondantes
   * Prédicats d'énumération classiques (entiers naturels, entiers relatifs, couple, puissance, produit cartésien, liste, partie, permutation)
   * Enumération + Contraintes => Recherche combinatoire 
5) Contrôle de l'exécution : Coupure, ordre supérieur, gel, assert
   * Coupure (mécanisme de base, not, coupure verte vs coupure rouge)

<img src="logoUGA.png"></img>

# Réversibilité

Un programme sera dit réversible si l'exécution prévue (Données ==>> Résultats) peut être inversé (Résultats <<== Données), c'est à dire si le profil informel du prédicat est stricte (certaines variables ont un statut de données en entrée strict, ou pas)

La réversibilité est établi en assurant la terminaison des exécutions pour des profils d'appels différents. En première approche, la correction et la complétude ne sont pas touchées.

La réversibilité n'est pas un objectif en soi, mais permet des usages plus diversifiés, réduit le nombre de programmes nécessaires (un seul programme pour la concaténation et la séparation, par ex.), ouvre des perspectives de recherche d'algorithmes nouveaux (en chaînage arrière, plutôt qu'en chaînage avant seul ; ou avec des chaînages mixtes), permet des utilisations avec des données partiellement connues (données imparfaites ou données avec parties manquantes) ; pour le test (avec les contraintes et le non-déterminisme, cf. plus loin), cela peut permettre de vérifier des propriétés par vérification de contrainte, et de produire des générateurs de tests automatiques.  

## Prédicats symétriques

Certains prédicats semblent symétriques, p(I,O) = p(O,I), pour autant les programmes sont-ils reversibles, est-il possible de les rendre réversibles ?
* symétrique (ok)

In [1]:
%%writefile prog.pl
/* symétrique(A,B) vrai ssi A et B sont 2 arbres binaires symétriques l'un de l'autre */
symetrique([],[]).
symetrique([Rac,FilsGaucheA,FilsDroitA],[Rac,FilsGaucheB,FilsDroitB]):-
  symetrique(FilsGaucheA,FilsDroitB),
  symetrique(FilsDroitA,FilsGaucheB).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [2]:
!echo "symetrique([1,[2,[],[]],[]],X)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [symetrique([1,[2,[],[]],[]],[1,[],[2,[],[]]])]


In [3]:
!echo "symetrique(X,[1,[2,[],[]],[]])." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [symetrique([1,[],[2,[],[]]],[1,[2,[],[]],[]])]


* inverser premier et dernier élément d'une liste (?, cela dépend de l'écriture)

In [5]:
%%writefile prog.pl
/* inversePermierDernier(A,B) vrai ssi A et B sont 2 avec les mêmes éléments sauf les extrèmes qui sont inversés */
inversePermierDernier([],[]).
inversePermierDernier([A],[A]).
inversePermierDernier([A,B],[B,A]).
inversePermierDernier(A,B) :-
  concatene([Premier],FinA,A),
  concatene([Dernier],FinB,B),
  concatene(Centre,[Dernier],FinA),
  concatene(Centre,[Premier],FinB).

concatene([],F,F).
concatene([E|L],F,[E|R]):-concatene(L,F,R).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [6]:
!echo "inversePermierDernier([1,2,3,4,5],X)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [inversePermierDernier([1,2,3,4,5],[5,2,3,4,1])]


In [7]:
!echo "inversePermierDernier(X,[1,2,3,4,5])." | swipl -g halt -s prog.pl

Votre requête ?

ERROR: Stack limit (1.0Gb) exceeded
ERROR:   Stack sizes: local: 4Kb, global: 0.9Gb, trail: 0Kb
ERROR:   Stack depth: 19,173,120, last-call: 100%, Choice points: 13
ERROR:   In:
ERROR:     [19,173,120] user:concatene('<garbage_collected>', [length:1], _416)
ERROR:     [38] user:inversePermierDernier('<garbage_collected>', '<garbage_collected>')
ERROR:     [37] '$bags':findall_loop(<compound inversePermierDernier/2>, '<garbage_collected>', _464, [])
ERROR:     [36] system:setup_call_catcher_cleanup(<compound (:)/2>, <compound (:)/2>, _494, <compound (:)/2>)
ERROR:     [32] user:main
ERROR: 
ERROR: Use the --stack_limit=size[KMG] command line option or
ERROR: ?- set_prolog_flag(stack_limit, 2_147_483_648). to double the limit.


Pour comprendre pourquoi l'un fonctionne et pas l'autre, il faut analyser la réversibilité de la concaténation (puis la réversibilité du prédicat d'inversion premier-dernier, ensuite).

<img src="logoUGA.png"></img>

## Concaténation

Cas de la concaténation :

In [ ]:
%%writefile prog.pl
/* concatene(D,F,L) est vrai ssi la concaténation des listes D et F corespond à la liste L. */
concatene([],F,F).
concatene([E|L],F,[E|R]):-concatene(L,F,R).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

In [16]:
!echo "concatene([1,2],[3,4,5],L),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
L=[1,2,3,4,5]
Liste des résultats : [(concatene([1,2],[3,4,5],[1,2,3,4,5]),write(L=),writeln([1,2,3,4,5]))]


In [17]:
!echo "concatene(D,[3,4,5],[1,2,3,4,5]),write(\"D=\"),writeln(D)." | swipl -g halt -s prog.pl

Votre requête ?
D=[1,2]
Liste des résultats : [(concatene([1,2],[3,4,5],[1,2,3,4,5]),write(D=),writeln([1,2]))]


In [18]:
!echo "concatene([1,2],F,[1,2,3,4,5]),write(\"F=\"),writeln(F)." | swipl -g halt -s prog.pl

Votre requête ?
F=[3,4,5]
Liste des résultats : [(concatene([1,2],[3,4,5],[1,2,3,4,5]),write(F=),writeln([3,4,5]))]


In [19]:
!echo "concatene(D,F,[1,2,3,4,5]),write(\"D=\"),writeln(D),write(\"F=\"),writeln(F)." | swipl -g halt -s prog.pl

Votre requête ?
D=[]
F=[1,2,3,4,5]
D=[1]
F=[2,3,4,5]
D=[1,2]
F=[3,4,5]
D=[1,2,3]
F=[4,5]
D=[1,2,3,4]
F=[5]
D=[1,2,3,4,5]
F=[]
Liste des résultats : [(concatene([],[1,2,3,4,5],[1,2,3,4,5]),write(D=),writeln([]),write(F=),writeln([1,2,3,4,5])),(concatene([1],[2,3,4,5],[1,2,3,4,5]),write(D=),writeln([1]),write(F=),writeln([2,3,4,5])),(concatene([1,2],[3,4,5],[1,2,3,4,5]),write(D=),writeln([1,2]),write(F=),writeln([3,4,5])),(concatene([1,2,3],[4,5],[1,2,3,4,5]),write(D=),writeln([1,2,3]),write(F=),writeln([4,5])),(concatene([1,2,3,4],[5],[1,2,3,4,5]),write(D=),writeln([1,2,3,4]),write(F=),writeln([5])),(concatene([1,2,3,4,5],[],[1,2,3,4,5]),write(D=),writeln([1,2,3,4,5]),write(F=),writeln([]))]


In [ ]:
!echo "concatene(D,[3,4,5],L),write(\"D=\"),writeln(D),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

In [22]:
!echo "concatene([1,2],F,L),write(\"F=\"),writeln(F),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
F=_2486
L=[1,2|_2486]
Liste des résultats : [(concatene([1,2],_2588,[1,2|_2588]),write(F=),writeln(_2588),write(L=),writeln([1,2|_2588]))]


## Prédicats courants

* sur le premier élément d'une liste : premier, fin, ajoutEnPremier, supprimePremier, séparePremierFin
* sur le dernier élément d'une liste : dernier, debut, ajoutEnFin, sépareDebutDernier
* prédicat linéaire sur une liste donnant un nombre : longueur, somme, indice, nbOccurence
* prédicat linéaire sur une liste donnant une liste : remplace, filtre, concatene, supprDoublonConsécutif
* prédicat moins élémentaire sur une liste : inversePremierDernier, inverse, supprDoublonQlcq, tri

On peut étudier maintenant et comprendre la reversibilité de inversePremierDernier.

Parmi les prédicats de la premier liste, ajoutEnPremier et supprimePremier sont inverse l'un de l'autre et peuvent être fait à partir de séparePremierFin s'il est réversible (ce qui semble raisonnable, le prédicat ne nécessite probablement pas de récursivité, la terminaison ne doit pas être un problème).

In [29]:
%%writefile prog.pl
/* separePremierFin(E,F,L) est vrai ssi E est le premier et F la fin de la liste L, i.e. : [E|F] = L . */
separePremierFin(E,F,[E|F]).

/* ajoutEnPremier(E,L,R) est vrai ssi en ajoutant E en premier à L , on a la même liste que R */
ajoutEnPremier(E,L,Res) :-
  separePremierFin(E,L,Res).

/* supprimePremier(L,R) est vrai ssi en supprimant le premier élément de L , on a la même liste que R */
supprimePremier(L,Res) :-
  separePremierFin(_E,Res,L).
    
main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [26]:
!echo "ajoutEnPremier(0,[1,2,3],L),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
L=[0,1,2,3]
Liste des résultats : [(ajoutEnPremier(0,[1,2,3],[0,1,2,3]),write(L=),writeln([0,1,2,3]))]


In [27]:
!echo "supprimePremier([1,2,3],L),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
L=[2,3]
Liste des résultats : [(supprimePremier([1,2,3],[2,3]),write(L=),writeln([2,3]))]


Pour les prédicats sur le dernier élément, est-il raisonnable de vouloir le même comportement ? Pour le premier élément, il n'y avait pas de difficulté (car pas de récursivité), mais pour le dernier élément, à priori cela sera plus difficile à cause de la récursivité et du risque de non-terminaison. Il faut donc vérifier la terminaison avec attention

In [30]:
%%writefile prog.pl
/* separeDebutDernier(Debut,Dernier,L) est vrai ssi Debut est le début et Dernier le dernier élément de la liste L */
separeDebutDernier([],Dernier,[Dernier]).
separeDebutDernier([E|Debut],Dernier,[E|Res]):-
  separeDebutDernier(Debut,Dernier,Res).

/* ajoutEnDernier(E,L,R) est vrai ssi en ajoutant E en dernier à L , on a la même liste que R */
ajoutEnDernier(E,L,Res) :-
  separeDebutDernier(L,E,Res).

/* supprimeDernier(L,R) est vrai ssi en supprimant le dernier élément de L , on a la même liste que R */
supprimeDernier(L,Res) :-
  separeDebutDernier(Res,_E,L).
    
main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [31]:
!echo "ajoutEnDernier(4,[1,2,3],L),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
L=[1,2,3,4]
Liste des résultats : [(ajoutEnDernier(4,[1,2,3],[1,2,3,4]),write(L=),writeln([1,2,3,4]))]


In [32]:
!echo "supprimeDernier([1,2,3],L),write(\"L=\"),writeln(L)." | swipl -g halt -s prog.pl

Votre requête ?
L=[1,2]
Liste des résultats : [(supprimeDernier([1,2,3],[1,2]),write(L=),writeln([1,2]))]


Pour les autres prédicats, idem, il faut regarder finement les terminaisons et aussi, parfois, se demander ce que pourrait être l'inverse d'un prédicat. Ex. :

* l'inverse de la longueur, la génération d'une liste quelconque de longueur donnée ? (ok)
* l'inverse de la somme, en limitant aux entiers positifs non nul, l'ensemble des découpages possibles d'un entier en somme d'entiers positifs non nuls ; en ne supposant pas de limitation, ... ? 
* l'inverse du tri, la permutation ? (ok, mais il y a peut-être plus simple)
Visiblement, certains prédicats risquent de ne pas être facilement inversibles.

La réversibilité, introduit deux questions naturelles : comment assurer l'inversion numérique (de A+B => = C, peut-on déduire à partir de A et de C la valeur de B) et comment caractériser/prendre en compte la multiplicité des solutions (ex. les permutations).

<img src="logoUGA.png"></img>

# Non déterminisme

Avec les prédicats réversibles ou naturellement avec certains prédicats, plusieurs solutions peuvent être trouvées par le moteur d'inférence ProLog. 

In [61]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E|_L]).
membre(E,[_F|L]):-
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [62]:
!echo "membre(E,[1,2,3]), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

Votre requête ?
E=1
E=2
E=3
Liste des résultats : [(membre(1,[1,2,3]),write(E=),writeln(1)),(membre(2,[1,2,3]),write(E=),writeln(2)),(membre(3,[1,2,3]),write(E=),writeln(3))]


Ces solutions :

* peuvent être redondantes (c'est complet, mais redondant ; en travaillant sur la complétude, sans la perdre, l'objectif est alors de ne plus avoir de règles redondantes)
* peuvent être en nombre infini (alors il faut prévoir un mécanisme de coupure, pour éviter une boucle infinie, ou se restreindre à la première solution
  * parmi les solutions en nombre infini, attention à garantir que toutes les solutions arrivent si l'on attends assez longtemps.
 
Exemple de solution redondante (pour membre) :

In [71]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E]).
membre(E,[E|_L]).
membre(E,[_F|L]):-
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [72]:
!echo "membre(E,[1,2,3]), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

Votre requête ?
E=1
E=2
E=3
E=3
Liste des résultats : [(membre(1,[1,2,3]),write(E=),writeln(1)),(membre(2,[1,2,3]),write(E=),writeln(2)),(membre(3,[1,2,3]),write(E=),writeln(3)),(membre(3,[1,2,3]),write(E=),writeln(3))]


attention à ne pas réduire la complétude en voulant éviter la redondance

In [73]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E]).
membre(E,[_F|L]):-
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [74]:
!echo "membre(E,[1,2,3]), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

Votre requête ?
E=3
Liste des résultats : [(membre(3,[1,2,3]),write(E=),writeln(3))]


Autre soucis potentiel, (avec le programme initial), il peut y avoir redondance des réponses, si un élément apparait plusieurs fois dans la liste.  
Pour éviter cela :

In [116]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E|_L]).
membre(E,[F|L]):-
  dif(E,F),
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [117]:
!echo "membre(E,[1,2,1]), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

Votre requête ?
E=1
E=2
Liste des résultats : [(membre(1,[1,2,1]),write(E=),writeln(1)),(membre(2,[1,2,1]),write(E=),writeln(2))]


L'exemple précédent, a limiter les inconnues en fixant la liste, et demandant l'élément.  
Pour aller plus loin, on va mettre en inconnu, l'élément et la liste.  
On repart du prédicat initial.

In [113]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E|_L]).
membre(E,[_F|L]):-
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [ ]:
!echo "membre(E,L), write(\"E=\"), write(E), write(\" L=\"), writeln(L)." | swipl -g halt -s prog.pl

réponse :
```
E=_2528 L=[_2528|_2624]
E=_2528 L=[_2622,_2528|_2630]
E=_2528 L=[_2622,_2628,_2528|_2636]
E=_2528 L=[_2622,_2628,_2634,_2528|_2642]
E=_2528 L=[_2622,_2628,_2634,_2640,_2528|_2648]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2528|_2654]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2652,_2528|_2660]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2652,_2658,_2528|_2666]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2652,_2658,_2664,_2528|_2672]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2652,_2658,_2664,_2670,_2528|_2678]
E=_2528 L=[_2622,_2628,_2634,_2640,_2646,_2652,_2658,_2664,_2670,_2676,_2528|_2684]
...
```

Inconvénient : cela ne donne pas de liste explicite (la fin de liste est implicite)  
Pour améliorer cela, on peut demander à expliciter la fin de liste, avec un prédicat qui enumère les listes.

In [120]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E|L]):-
  liste(L).
membre(E,[_F|L]):-
  membre(E,L).

liste([]).
liste([_E|L]):-
  liste(L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [ ]:
!echo "membre(E,L), write(\"E=\"), write(E), write(\" L=\"), writeln(L)." | swipl -g halt -s prog.pl

Réponse :
```
E=_2600 L=[_2600]
E=_2600 L=[_2600,_2700]
E=_2600 L=[_2600,_2700,_2706]
E=_2600 L=[_2600,_2700,_2706,_2712]
E=_2600 L=[_2600,_2700,_2706,_2712,_2718]
E=_2600 L=[_2600,_2700,_2706,_2712,_2718,_2724]
E=_2600 L=[_2600,_2700,_2706,_2712,_2718,_2724,_2730]
E=_2600 L=[_2600,_2700,_2706,_2712,_2718,_2724,_2730,_2736]
E=_2600 L=[_2600,_2700,_2706,_2712,_2718,_2724,_2730,_2736,_2742]
```
C'est mieux, les listes sont explicites, mais, les listes ont toutes E en début, aucune liste avec E en second, ou en 3e position.  
C'est qu'il y a 2 énumarations infinie concurrentes, toutes les listes avec E, et toutes les listes de fin (qlc).  
C'est un problème courant (général).  
On peut trouver une demi-solution en limitant la taille de la liste résultat.

In [123]:
!echo "membre(E,[A,B,C]), write(\"E=\"), write(E), write(\" L=\"), writeln([A,B,C])." | swipl -g halt -s prog.pl

Votre requête ?
E=_2602 L=[_2602,_2608,_2614]
E=_2608 L=[_2602,_2608,_2614]
E=_2614 L=[_2602,_2608,_2614]
Liste des résultats : [(membre(_2940,[_2940,_2952,_2958]),write(E=),write(_2940),write( L=),writeln([_2940,_2952,_2958])),(membre(_2840,[_2846,_2840,_2858]),write(E=),write(_2840),write( L=),writeln([_2846,_2840,_2858])),(membre(_2740,[_2746,_2752,_2740]),write(E=),write(_2740),write( L=),writeln([_2746,_2752,_2740]))]


Il faudrait pouvoir faire une requête, avec liste initiale de longueur 1, puis 2, puis 3, etc.  
Pour faire cela (c'est une solution générale), on change la requête initiale (seulement la requête initiale, cela ne change pas le programme spécifique, mais répond à la recherche d'une requête qui demande des listes de longueur 1 puis 2, puis 3, etc.). Cela utilise aussi le prédicat ```liste```, c'est un hazard, mais c'est bien le même prédicat :

In [ ]:
!echo "liste(L), membre(E,L), write(\"E=\"), write(E), write(\" L=\"), writeln(L)." | swipl -g halt -s prog.pl

Réponse :
```
Votre requête ?
E=_2604 L=[_2604]
E=_2604 L=[_2604,_2710]
E=_2604 L=[_2704,_2604]
E=_2604 L=[_2604,_2710,_2716]
E=_2604 L=[_2704,_2604,_2716]
E=_2604 L=[_2704,_2710,_2604]
E=_2604 L=[_2604,_2710,_2716,_2722]
E=_2604 L=[_2704,_2604,_2716,_2722]
E=_2604 L=[_2704,_2710,_2604,_2722]
E=_2604 L=[_2704,_2710,_2716,_2604]
E=_2604 L=[_2604,_2710,_2716,_2722,_2728]
E=_2604 L=[_2704,_2604,_2716,_2722,_2728]
E=_2604 L=[_2704,_2710,_2604,_2722,_2728]
```
C'est ok ! On voit bien l'élément E apparaitre explicitement dans toutes les listes, avec des listes de toutes les longueurs. 
Cependant, on pourrait vouloir avoir 2 variantes, l'une où on explicite les E (et quand c'est une autre variable, ce ne peut pas être E), et l'une ou E est unique dans la liste.   
Avec E multiples explicites :

In [129]:
%%writefile prog.pl
/* membre(E,L) est vrai ssi E est dans la liste L */
membre(E,[E|L]):-
  listeSansE(E,L).
membre(E,[E|L]):-
  membre(E,L).
membre(E,[F|L]):-
  dif(E,F),
  membre(E,L).

listeSansE(_E,[]).
listeSansE(E,[F|L]):-
  dif(E,F),
  listeSansE(E,L).

liste([]).
liste([_E|L]):-
  liste(L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [ ]:
!echo "liste(L), membre(E,L), write(\"E=\"), write(E), write(\" L=\"), writeln(L)." | swipl -g halt -s prog.pl

Réponse :
```
E=_2620 L=[_2620]
E=_4520 L=[_4520,_4552]
E=_2620 L=[_2620,_2620]
E=_4520 L=[_4552,_4520]
E=_4526 L=[_4526,_4558,_4678]
E=_4526 L=[_4526,_4526,_4558]
E=_2620 L=[_2620,_2620,_2620]
E=_4526 L=[_4526,_4558,_4526]
E=_4526 L=[_4558,_4526,_4680]
E=_4526 L=[_4558,_4526,_4526]
E=_4526 L=[_4558,_4678,_4526]
E=_4532 L=[_4532,_4564,_4684,_4804]
E=_4532 L=[_4532,_4532,_4564,_4684]
E=_4532 L=[_4532,_4532,_4532,_4564]
E=_2620 L=[_2620,_2620,_2620,_2620]
E=_4532 L=[_4532,_4532,_4564,_4532]
E=_4532 L=[_4532,_4564,_4532,_4686]
E=_4532 L=[_4532,_4564,_4532,_4532]
E=_4532 L=[_4532,_4564,_4684,_4532]
E=_4532 L=[_4564,_4532,_4686,_4808]
E=_4532 L=[_4564,_4532,_4532,_4686]
E=_4532 L=[_4564,_4532,_4532,_4532]
E=_4532 L=[_4564,_4532,_4686,_4532]
E=_4532 L=[_4564,_4684,_4532,_4806]
E=_4532 L=[_4564,_4684,_4532,_4532]
E=_4532 L=[_4564,_4684,_4804,_4532]
```
C'est ok, pour vérifier qu'il y a le bon nombre de réponses, il faut faire un peu de combinatoire. Il y a à chaque fois $2^n-1$ listes (en binaire, tous les nombres de n bits, sauf 00...0).  
Remarque, c'est une bonne pratique de faire de la combinatoire pour vérifier que l'on a le bon nombre de solutions.

Pour, E unique (ce n'est plus membre, mais membreUnique) :

In [131]:
%%writefile prog.pl
/* membreUnique(E,L) est vrai ssi E est dans la liste L */
membreUnique(E,[E|L]):-
  listeSansE(E,L).
membreUnique(E,[F|L]):-
  dif(E,F),
  membreUnique(E,L).

listeSansE(_E,[]).
listeSansE(E,[F|L]):-
  dif(E,F),
  listeSansE(E,L).

liste([]).
liste([_E|L]):-
  liste(L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [ ]:
!echo "liste(L), membreUnique(E,L), write(\"E=\"), write(E), write(\" L=\"), writeln(L)." | swipl -g halt -s prog.pl

Réponse :
```
E=_2620 L=[_2620]
E=_4520 L=[_4520,_4552]
E=_4520 L=[_4552,_4520]
E=_4526 L=[_4526,_4558,_4678]
E=_4526 L=[_4558,_4526,_4680]
E=_4526 L=[_4558,_4678,_4526]
E=_4532 L=[_4532,_4564,_4684,_4804]
E=_4532 L=[_4564,_4532,_4686,_4808]
E=_4532 L=[_4564,_4684,_4532,_4806]
E=_4532 L=[_4564,_4684,_4804,_4532]
```

Remarques :

* ce ne serait pas une bonne idée d'intégrer l'appel de la requête à liste dans un prédicat qui ferait cet appel.  
Ponctuellement cela marcherait, si on associait plus tard plusieurs prédicats de ce type, on risquerait de retomber sur le problème intial (plusieurs énumérations infinie en parallèle).  
Cette solution fonctionne, parce que l'appel est le premier de la requête, et contraint toutes les énumérations à suivre.
* cette solution, n'est pas très efficace, pour chaque nouvelle liste initiale créée par l'appel du premier liste, toutes les énumération sont reproduites.
Pour une énumérations initialement "linéaire", on passe à une complexité "quadratique"  


<img src="logoUGA.png"></img>

## Enumérations classiques

Parmi les prédicats non déterministes, on trouve tous les prédicats d'énumérations d'ensembles (finis ou pas) :

* énumération d'un ensemble fini (membre, intervalle)
* énumération d'un ensemble infini (entier naturel, entier pair, entier relatif, etc.)
* énumération d'un produit cartésien (ex. : couple de booléens, trio de nombres, etc.), d'une liste d'entier
* énumération des parties d'un ensemble fini, d'une permutation

Un point commun à la plupart de ces programmes, les récursivités ne s'écrivent pas en 2 cas (1 cas de base, 1 cas de propagation ; sinon cela ne serait pas non-déterministe), mais avec au moins 3 cas (parfois cela peut être un peu caché).

Pour vérifier que ces programmes sont corrects, souvent, cela peut être une bonne idée de compter le nombre de solution (et vérifier que c'est bien ce que la combinatoire devait donner)

### Exemples

**Pour les entiers relatifs**, attention à panacher positifs et négatifs.  
  Pour comprendre comme cela marche (ou pas), dessiner des arbres d'exécution.  
  Rem. : Dans tous les cas, on peut prouver la correction, la complétude et la terminaison.

In [134]:
%%writefile prog.pl
/* entierRelatif(N) est vrai ssi N est un entier relatif */
entierRelatif(0).
entierRelatif(N):-
  entierPositif(N).
entierRelatif(N):-
  entierNegatif(N).

entierPositif(1).
entierPositif(N):-
  entierPositif(M),{N=M+1}.

entierNegatif(-1).
entierNegatif(N):-
  entierNegatif(M),{N=M-1}.

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- use_module(library(clpq)), main.

Overwriting prog.pl


In [ ]:
!echo "entierPositif(E), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

réponse :
```
E=1
E=2
E=3
E=4
E=5
E=6
E=7
E=8
E=9
E=10
```

In [ ]:
!echo "entierNegatif(E), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

réponse :
```
E=-1
E=-2
E=-3
E=-4
E=-5
E=-6
E=-7
E=-8
E=-9
E=-10
```

In [ ]:
!echo "entierRelatif(E), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

réponse :
```
E=1
E=2
E=3
E=4
E=5
E=6
E=7
E=8
E=9
E=10
```

Pour éviter de n'avoir que la moitié des entiers relatifs il faut prévoir 2 solutions quand cela termine (une solution positive et une négative) :

In [139]:
%%writefile prog.pl
/* entierRelatif(N) est vrai ssi N est un entier relatif */
entierRelatif(N):-
  entierRelatif(0,N).

entierRelatif(N,N).
entierRelatif(M,N):-
  {N = -M}.
entierRelatif(M,N):-
  {P = M+1},
  entierRelatif(P,N).


main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- use_module(library(clpq)), main.

Overwriting prog.pl


In [ ]:
!echo "entierRelatif(E), write(\"E=\"), writeln(E)." | swipl -g halt -s prog.pl

réponse :
```
E=0
E=0
E=1
E=-1
E=2
E=-2
E=3
E=-3
E=4
E=-4
E=5
E=-5
```

C'est une bonne solution (linéaire).  
Il y aurait une autre solution (moins efficace, i.e. quadratique), en préfixant la requête initiale par un prédicat qui énumère les entiers (naturels) et génère ainsi une borne max pour que les énumérations à suivre des entiers positifs et négatifs puissent être contraints par cette borne et ne soit pas infinis. C'est le type de solution générale déjà utilisé pour membre dans la section précédente : limiter les résultats dans une boule, ou sur une sphére, dont le rayon (de la boule ou de la sphère) parcours progressivement les valeurs 0, 1, 2,... 

**Exemple pour Parties**

In [95]:
%%writefile prog.pl
/* partie(P,L) est vrai ssi P est une partie de L */
partie([],[]).
partie([E|P],[E|L]):-
  partie(P,L).
partie(P,[_E|L]):-
  partie(P,L).


main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [97]:
!echo "partie(P,[1,2,3]), write(\"P=\"), writeln(P)." | swipl -g halt -s prog.pl

Votre requête ?
P=[1,2,3]
P=[1,2]
P=[1,3]
P=[1]
P=[2,3]
P=[2]
P=[3]
P=[]
Liste des résultats : [(partie([1,2,3],[1,2,3]),write(P=),writeln([1,2,3])),(partie([1,2],[1,2,3]),write(P=),writeln([1,2])),(partie([1,3],[1,2,3]),write(P=),writeln([1,3])),(partie([1],[1,2,3]),write(P=),writeln([1])),(partie([2,3],[1,2,3]),write(P=),writeln([2,3])),(partie([2],[1,2,3]),write(P=),writeln([2])),(partie([3],[1,2,3]),write(P=),writeln([3])),(partie([],[1,2,3]),write(P=),writeln([]))]


**Exemple pour permutation**

In [98]:
%%writefile prog.pl
/* permutation(P,L) est vrai ssi P est une permutation de L */
permutation([],[]).
permutation([E|P],L):-
  membre(E,L,R),
  permutation(P,R).

membre(E,[E|L],L).
membre(E,[F|L],[F|R]):-
  membre(E,L,R).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [99]:
!echo "permutation(P,[1,2,3]), write(\"P=\"), writeln(P)." | swipl -g halt -s prog.pl

Votre requête ?
P=[1,2,3]
P=[1,3,2]
P=[2,1,3]
P=[2,3,1]
P=[3,1,2]
P=[3,2,1]
Liste des résultats : [(permutation([1,2,3],[1,2,3]),write(P=),writeln([1,2,3])),(permutation([1,3,2],[1,2,3]),write(P=),writeln([1,3,2])),(permutation([2,1,3],[1,2,3]),write(P=),writeln([2,1,3])),(permutation([2,3,1],[1,2,3]),write(P=),writeln([2,3,1])),(permutation([3,1,2],[1,2,3]),write(P=),writeln([3,1,2])),(permutation([3,2,1],[1,2,3]),write(P=),writeln([3,2,1]))]


<img src="logoUGA.png"></img>

# Contraintes

(fin du prolog pur : tout ce qui précède, dont reversibilité et non-déterminisme, (tout hors dif), est défini dans le noyau de prolog)  
(début d'un autre paradigme de programmation : la programmation logique avec contraintes, PLC/CLP qui étend la programmation logique et la programmation par contraintes, en réutilisant les avancées de la Recherche Opérationnelle RO)

Rappel : 

* en pur Prolog pas d'arithmétique (N+1 est un terme/arbre +(N,1))
* pour ajouter de l'arithmétique, pendant 20 ans, dif et is
  * dif : pour établir la différence entre termes
  * is : pour calculer des expressions

In [33]:
%%writefile prog.pl
main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [34]:
!echo "dif(2,3)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [dif(2,3)]


In [35]:
!echo "dif(2,2)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : []


In [36]:
!echo "dif(2,X)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [dif(2,_4610)]


In [37]:
!echo "dif(X,Y)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [dif(_4610,_4672)]


In [38]:
!echo "dif(X,X)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : []


* dif est reversible, il n'y a pas de statut spécifique pour les paramètres, toutes les exécutions fonctionnent. Avec dif, on peut faire un prédicat supprimeDoublon reversible, mais attention à ne pas en vouloir trop (ajouter des doublons, cela peut se faire un nombre infini de fois, il risque d'y avoir une nombre de solution infini ; pour limiter les réponses, donner une liste réponse de taille fixe, ex. : supprimeDoublon([A,B,C,D],[1,2]). )

In [39]:
!echo "E is 2+3." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [5 is 2+3]


In [40]:
!echo "5 is E+3." | swipl -g halt -s prog.pl

Votre requête ?

ERROR: is/2: Arguments are not sufficiently instantiated


* is n'est pas réversible, son utilisation dans les prédicats introduit des comportements non réversibles.

Rem. (il n'y a pas que is en cause, c'est tout l'arithmétique et les tests, l'utilisation prévue est orientée et non-réversible) :

In [41]:
!echo "2 > X." | swipl -g halt -s prog.pl

Votre requête ?

ERROR: >/2: Arguments are not sufficiently instantiated


Pour pallier à ce défaut de réversibilité pour l'arithmétique, de nombreuses solutions ont été explorées par la communauté prolog :

* contraintes pseudo-linéaires (simplexe)
* contraintes entières ou réelles, ou complexes
* contraintes sur ensemble fini, ou sur intervalle
* propagation de contraintes


Pour ce cours, le moteur de résolution adopté sera clpq, dit de résolution de contraintes rationnelles (pseudo-linéaire). Cela limitera, certes, mais en pratique cela sera suffisant (en particulier en Miage). La syntaxe est simple, il suffit d'ajouter des accolades, ex. : {A+B=C}

Le moteur de résolution de contrainte s'intègre au moteur d'unification, dans les arbres d'exécution (à la main), on pourra ajouter des ensembles de contraintes à respecter, et les résoudre "à la main", si nécessaire. 

In [42]:
%%writefile prog.pl
main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- use_module(library(clpq)), main.

Overwriting prog.pl


In [43]:
!echo "{5=E+3}." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [{5=2+3}]


D'autres domaines scientifique utilisant des contraintes (appartenant à la Recherche Operationnelle ou à la Programmation par Contraintes) ont aussi été mis à contribution pour donner ainsi naissance à la Programmation Logique avec Contraintes (PLC ou CLP) :

* contraintes sur les booléens ou sur des ensembles finis (pour des représentations et des énumérations "efficaces", ou pour des représentation de problème en 0-1, avec ou sans (ensembliste))
* contraintes structurelles (sur les structures arborescentes => graphes)
  * exemple de listes circulaires

In [44]:
!echo "L=[1,2,3|L]." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : @([S_1=S_1],[S_1=[1,2,3|S_1]])


  * exemple de graphe

In [47]:
!echo "L=[1,[2,L,[]],[3,[],L]]." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : @([S_1=S_1],[S_1=[1,[2,S_1,[]],[3,[],S_1]]])


Certains prédicats continuent de marcher sur ces objets (d'autres non, ou pas tel que) :

* premier, ajoutEnPremier, supprimePremier : ok
* dernier, etc. : ko

In [49]:
%%writefile prog.pl
/* separePremierFin(E,F,L) est vrai ssi E est le premier et F la fin de la liste L, i.e. : [E|F] = L . */
separePremierFin(E,F,[E|F]).

/* ajoutEnPremier(E,L,R) est vrai ssi en ajoutant E en premier à L , on a la même liste que R */
ajoutEnPremier(E,L,Res) :-
  separePremierFin(E,L,Res).

/* supprimePremier(L,R) est vrai ssi en supprimant le premier élément de L , on a la même liste que R */
supprimePremier(L,Res) :-
  separePremierFin(_E,Res,L).

/* separeDebutDernier(Debut,Dernier,L) est vrai ssi Debut est le début et Dernier le dernier élément de la liste L */
separeDebutDernier([],Dernier,[Dernier]).
separeDebutDernier([E|Debut],Dernier,[E|Res]):-
  separeDebutDernier(Debut,Dernier,Res).

/* ajoutEnDernier(E,L,R) est vrai ssi en ajoutant E en dernier à L , on a la même liste que R */
ajoutEnDernier(E,L,Res) :-
  separeDebutDernier(L,E,Res).

/* supprimeDernier(L,R) est vrai ssi en supprimant le dernier élément de L , on a la même liste que R */
supprimeDernier(L,Res) :-
  separeDebutDernier(Res,_E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [51]:
!echo "L=[1,2|L], supprimePremier(L,R)." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : @([([1|S_1]=[1|S_1],supprimePremier([1|S_1],S_1))],[S_1=[2,1|S_1]])


In [52]:
!echo "L=[1,2|L], ajoutEnDernier(0,L,R)." | swipl -g halt -s prog.pl

Votre requête ?

ERROR: Stack limit (1.0Gb) exceeded
ERROR:   Stack sizes: local: 3Kb, global: 0.9Gb, trail: 0Kb
ERROR:   Stack depth: 38,346,207, last-call: 100%, Choice points: 13
ERROR:   In:
ERROR:     [38,346,207] user:separeDebutDernier('<garbage_collected>', 0, _416)
ERROR:     [37] '$bags':findall_loop(<compound (',')/2>, '<garbage_collected>', _438, [])
ERROR:     [36] system:setup_call_catcher_cleanup(<compound (:)/2>, <compound (:)/2>, _468, <compound (:)/2>)
ERROR:     [32] user:main
ERROR:     [31] system:catch(<compound (:)/2>, <compound error/2>, <compound (:)/2>)
ERROR: 
ERROR: Use the --stack_limit=size[KMG] command line option or
ERROR: ?- set_prolog_flag(stack_limit, 2_147_483_648). to double the limit.


L'algorithmique de ces objets doit prendre en compte les boucles possibles dans les structures de données pour éviter les boucles infinies intr (c'est possible, en marquant ces objets ou en gardant la liste des objets déjà visités, cf. algorithmique sur les graphes)

<img src="logoUGA.png"></img>

# Non-déterminisme + Contraintes = résolution combinatoire, contraintes en premier

L'association des contraintes et du non-déterminisme donne une programmation combinatoire (parfois efficace) pour la résolution de problèmes ; souvent, si elle n'est pas efficace, elle est au moins simple à mettre en oeuvre.

La démarche classique :

```prolog
probleme(donnees,solution):-
  enumération(donnees, candidatsSolutions),
  vérificationContraintes(candidatsSolutions,solution).
```

est remplacée par une résolution combinatoire, contraintes en premier :

```prolog
probleme(donnees,solution):-
  vérificationContraintes(candidatsSolutions,solution),
  enumération(donnees, candidatsSolutions).
```

Exemple, coloriage de la carte de la Suisse (et pays limitrophes) :

* le problème : 4 couleurs, 5 pays, éviter de colorier les pays voisins avec la même couleur.
* solution classique : énumération des 4\*4\*4\*4\*4 (1024) cartes possibles, et pour chaque carte vérifications de contraintes de proximité (en général, les premiers coloriages sont uniformes, et pour sortir des solutions uniformes, cela prends du temps)
* solution aléatoire (pas si mauvaise, mais aléatoire)
* solutions combinatoire, contraintes en premier : mettre en place les contraintes se fait en une 8 étapes (une fois pour toute), l'énumération des couleurs évite les coloriages uniformes, c'est plus rapide

# Réversibilité + Non-déterminisme + Contraintes ==>> validation de programme (preuves et tests), une piste à explorer ?

* preuve, ex : ?- estTrié([1,2,X,Y,3,4]). ?- estTrié([3,4,X,Y,1,2]). ?- estTrié([X,Y,Z]). *à explorer*
* tests, générateur de tests, similaire à résolution combinatoire, mais sous forme tests combinatoires avec vérification des tests à suivre. *à explorer*

# Contrôle de l'exécution : Coupure, ordre supérieur, gel, assert

## Coupure : !

* mécanisme de base avec un arbre
* not ou \+ :  
  `not(P):= P, !, fail.`  
  `not(P).`  
  mais attention au not(not(P)), ce n'est pas équivalent à P
* définition des coupure verte (ne réduit pas l'espace de solution, ex. une coupure après la requête pour une recherche de la premier solution) et des coupure rouge (peut réduire l'espace des solutions)

In [105]:
%%writefile prog.pl
membre(E,[E|_L]).
membre(E,[_F|L]):-
  membre(E,L).

main :-  writeln('Votre requête ?'), read(Entree),
  findall(Entree,Entree,Resultats),
  write('Liste des résultats : '), writeln(Resultats).
:- main.

Overwriting prog.pl


In [106]:
!echo "not(membre(1,[2,0,2,4]))." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [not(membre(1,[2,0,2,4]))]


In [107]:
!echo "not(membre(0,[2,0,2,4]))." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : []


In [109]:
!echo "not(membre(X,[2,0,2,4]))." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : []


In [110]:
!echo "not(not(membre(X,[2,0,2,4])))." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [not(not(membre(_2612,[2,0,2,4])))]


In [111]:
!echo "membre(X,[2,0,2,4])." | swipl -g halt -s prog.pl

Votre requête ?
Liste des résultats : [membre(2,[2,0,2,4]),membre(0,[2,0,2,4]),membre(2,[2,0,2,4]),membre(4,[2,0,2,4])]


## Autres

* Ordre supérieur (c'est findAll, call, apply : déjà vu ou utilisé ; peut permettre l'introspection)
* Gel (c'est une manière de gérer les contraintes : un prédicat est gelé sur une variable, le moteur attend tant que la variable est inconnu, quand la variable est connue, l'exécution du prédicat est lancée)
* Assert (on ajoute des règles au programme par programme), cela peut donner naissance à des optimisations, et à une forme de programmation dite programmation dynamique (avec mémoïsation)